In [1]:
import json

import nltk
import pandas as pd
import torch

from model import LM
from utils import transliterate

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokens = json.load(open("tokenizer.json", "r"))


lm = LM()

test = pd.read_csv("data/test.csv")

epochs = range(10, 110, 10)

for epoch in epochs:
    checkpoint = torch.load(
        f"checkpoints/epoch_{epoch}.pt",
        weights_only=True,
        map_location=device
    )

    lm.load_state_dict(checkpoint['lm_state_dict'])
    lm.eval()
    lm.to(device)

    avg_dist = 0

    for bn, en in test.values:
        translit = transliterate(bn, lm, tokens)

        dist = nltk.edit_distance(en, translit) / len(en)
        avg_dist += dist

    print("Epoch:", epoch, avg_dist / len(test))

Epoch: 10 0.18569786152168014
Epoch: 20 0.1800420610041972
Epoch: 30 0.16699999066413598
Epoch: 40 0.1632056169372563
Epoch: 50 0.16709169230229556
Epoch: 60 0.16791787065988636
Epoch: 70 0.16544667335287308
Epoch: 80 0.16693759064544186
Epoch: 90 0.16496630974389742
Epoch: 100 0.16702055291807172


In [3]:
best_epoch = 40
checkpoint = torch.load(
    f"checkpoints/epoch_{best_epoch}.pt",
    weights_only=True,
    map_location=device
)

lm.load_state_dict(checkpoint['lm_state_dict'])
lm.eval()

text = "এমন দিনে বিনা কাজের অবকাশে বিনয়ভূষণ তাহার বাসার দোতলার বারান্দায় একলা দাঁড়াইয়া রাস্তায় জনতার চলাচল দেখিতেছিল।"

for t in text.split(" "):
    print(t, transliterate(t, lm, tokens))

এমন emon
দিনে dine
বিনা bina
কাজের kajer
অবকাশে abokashe
বিনয়ভূষণ binoybhushan
তাহার tahar
বাসার basar
দোতলার dotolar
বারান্দায় baranday
একলা ekla
দাঁড়াইয়া daraiya
রাস্তায় rastay
জনতার jontar
চলাচল cholachol
দেখিতেছিল। dekhitechilo
